In [1]:
import pandas as pd
import numpy as np
import random as rnd
from termcolor import colored

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

bold_start = '\033[1m'
underline_start = '\033[4m'
bu_end = '\033[0m'


Reading the data and understading the types

In [2]:
data = pd.read_csv('C:/Users/talro/OneDrive/Documents/death_prediction_synthetic.csv')
data.describe()

,age,sex,ses,residence_cd,weigh,heigh,BMI,bp_sys,bp_dias,smoking,...,framingham_cvd,antidiabetics,ERD,CVD,HTN,cancer,cardiovascular_meds,statines,immigrant,dead_5y
count,12438.000000,12438.000000,12438.000000,12438.000000,12438.000000,12438.000000,12438.000000,12438.000000,12438.000000,12438.000000,...,12438.000000,12438.000000,12438.000000,12438.000000,12438.000000,12438.0,12438.000000,12438.000000,12438.000000,12438.000000
mean,43.501126,1.468805,1.677279,1.062631,89.837746,167.943906,31.871922,124.803988,77.315163,1.636678,...,0.127533,0.857694,0.006593,0.092057,0.318058,0.0,0.602669,0.702203,0.174867,0.015839
std,4.802035,0.499046,0.763641,0.242307,19.296247,10.068620,6.459714,13.615760,8.915362,0.871735,...,0.092716,0.349378,0.080931,0.289117,0.465741,0.0,0.489365,0.457308,0.379869,0.124856
min,30.000000,1.000000,1.000000,1.000000,41.000000,91.500000,15.790000,75.000000,40.000000,1.000000,...,0.007310,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,41.000000,1.000000,1.000000,1.000000,76.000000,160.000000,27.400000,118.000000,70.000000,1.000000,...,0.061800,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,45.000000,1.000000,1.000000,1.000000,88.000000,168.000000,31.050000,124.000000,79.000000,1.000000,...,0.102370,1.000000,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,0.000000,0.000000
75%,48.000000,2.000000,2.000000,1.000000,100.000000,175.000000,35.350000,130.000000,80.000000,3.000000,...,0.163960,1.000000,0.000000,0.000000,1.000000,0.0,1.000000,1.000000,0.000000,0.000000
max,49.000000,2.000000,3.000000,2.000000,203.000000,202.000000,95.550000,200.000000,135.000000,3.000000,...,0.802220,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000


In [3]:
numberOfPeople = data.shape[0]
totalDeath = data["dead_5y"].sum()

In [7]:
old_death_population = {
    2018: (1770, 2229702),
    2017: (1780, 2191646),
    2016: (1798, 2150096),
    2015: (1697, 2110721),
    2014: (1733, 2070562),
    2013: (1694, 2030785),
    2012: (1699, 1990188),
    2011: (1704, 1952624),
    }
print(f'{"Year":<8s} | {"Death":<10s} | {"Population":<10s} | {"DeathRate":<10s}')
for year in old_death_population.keys():
    death_rate_s = round((old_death_population[year][0]/old_death_population[year][1])*100, 4)
    print(f'{year:<8} | {old_death_population[year][0]:<10} | {old_death_population[year][1]:<10} | {death_rate_s:<10}%')

print(bold_start + underline_start+  f'{"Data":<8} | {totalDeath:<10} | {numberOfPeople:<10} | {str(round(totalDeath/numberOfPeople*100, 4)):<10}%' + bu_end)


Year     | Death      | Population | DeathRate 
2018     | 1770       | 2229702    | 0.0794    %
2017     | 1780       | 2191646    | 0.0812    %
2016     | 1798       | 2150096    | 0.0836    %
2015     | 1697       | 2110721    | 0.0804    %
2014     | 1733       | 2070562    | 0.0837    %
2013     | 1694       | 2030785    | 0.0834    %
2012     | 1699       | 1990188    | 0.0854    %
2011     | 1704       | 1952624    | 0.0873    %
Data     | 197        | 12438      | 1.5839    %
